# Setup

In [ ]:
from optibook.synchronous_client import Exchange

import time
import logging
logger = logging.getLogger('client')
logger.setLevel('ERROR')

print("Setup was successful.")

# Connecting to the Exchange

In [ ]:
# create a connection to the exchange
exchange = Exchange()
_ = exchange.connect()

# Getting market information

### Instrument Details

In [ ]:
# Load all instruments (products) available on the exchange
instruments = exchange.get_instruments()
instruments

{'MKT3': Instrument(instrument_id=MKT3, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...),
 'MKT2': Instrument(instrument_id=MKT2, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...),
 'MKT1': Instrument(instrument_id=MKT1, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...),
 'TEST': Instrument(instrument_id=TEST, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...),
 'DEMO': Instrument(instrument_id=DEMO, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...)}

In [ ]:
# The returned type of exchange.get_instruments() is a dictionary, select a single instrument
instrument = instruments['DEMO']
print(instrument)

Instrument(instrument_id=DEMO, tick_size=0.1, instrument_type=InstrumentType.STOCK, ...)


In [ ]:
# From any instruments we can get a lot of data:
print(instrument.instrument_id)
print(instrument.instrument_type)
print(instrument.tick_size)

# For particular types of instruments, additional fields are populated (such as e.g. the option expiry, strike and kind, or the future expiry)
# For a STOCK those fields are equal to None
print(instrument.expiry)

print(instrument.option_kind)
print(instrument.strike)
print(instrument.base_instrument_id)

DEMO
InstrumentType.STOCK
0.1
None
None
None
None


In [ ]:
# The InstrumentType is an Enum (enumerable) type, that means its value is one of a few supported categories.
# We can use this Enum to compare which specific type of Instrument we are dealing with.

# First import the InstrumentType enum (we would normally do this at the top of the file)
from optibook.common_types import InstrumentType

# Then make a comparison as such:
if instrument.instrument_type == InstrumentType.STOCK:
    print('It is a STOCK.')
elif instrument.instrument_type == InstrumentType.STOCK_OPTION:
    print('It is a STOCK_OPTION.')
elif instrument.instrument_type == InstrumentType.STOCK_FUTURE:
    print('It is a STOCK_FUTURE.')
else:
    print(f'It was another instrument_type: {instrument.instrument_type}.')

It is a STOCK.


In [ ]:
# Define a variable storing the instrument_id of the product we are interested in
instrument_id = 'DEMO'

### Order Book

In [ ]:
# The exchange.get_last_price_book() method returns a large compound datatype storing bids and offers, with price and volumes, and a timestamp. 
book = exchange.get_last_price_book(instrument_id)
book

PriceBook(DEMO 2022-08-08 14:06:03.405511)
 #bids | price | #asks 
       |  87.4 |  500  
       |  87.2 |  250  
  250  |  86.8 |       
  500  |  86.6 |       

In [ ]:
# How do we take it apart? First, extract one side of the order book, the bids, say
bids = book.bids
bids

[PriceVolume(price=86.80000000000001, volume=250),
 PriceVolume(price=86.60000000000001, volume=500)]

In [ ]:
# That's a list of PriceVolume combinations, which makes sense, one line of the order book is always a price with corresponding volume and we have multiple such orders
# Let's find the most competitive order, which is always the first entry in the list
best_bid = bids[0]
best_bid

PriceVolume(price=86.80000000000001, volume=250)

In [ ]:
# That's still a compound of price and volume, we can simply select which one we want to see to finally end up with a numeric type
print(best_bid.price)
print(best_bid.volume)

86.80000000000001
250


In [ ]:
# Putting it all together we can also write this. 
book.bids[0].price

86.80000000000001

In [ ]:
# Note you will see an error on the above line if there are no bids at all available (the list will have no entry 0)
# Better is to check first
if book.bids:
    print(book.bids[0].price)
else:
    print('No bids in the order book at all.')

86.80000000000001


## Orders

### Inserting

In [ ]:
# Insert bid LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
# The returned value is a 'InsertOrderResponse' which, if successfull, contains the order_id of the order
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='limit')

In [ ]:
# Insert ask LIMIT order - this trades against any current orders, and any remainders become new resting orders in the book
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='limit')

In [ ]:
# Insert bid IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=10, volume=5, side='bid', order_type='ioc')

In [ ]:
# Insert ask IOC order - this order trades against any resting volume which has an equal or better price, but does not remain in the 
# book if any volume is unfilled
exchange.insert_order(instrument_id, price=20, volume=5, side='ask', order_type='ioc')

### Deleting & amending

In [ ]:
# Load a list of own outstanding/resting orders
exchange.get_outstanding_orders(instrument_id)

In [ ]:
# Delete an order by order_id, the returned DeleteOrderRequest contains a boolean showing whether the delete was succesful
exchange.delete_order(instrument_id, order_id=1234)

In [ ]:
# Delete all outstanding orders for an instrument
exchange.delete_orders(instrument_id)

In [ ]:
# Change (amend) the volume of an existing resting order by order_id, the returned AmendOrderResponse contains a boolean showing whether the amend was succesful
exchange.amend_order(instrument_id, order_id=1234, volume=30)

### Public tradeticks

In [ ]:
# Load and store a list of all public tradeticks since the instantiation of the Exchange (upto a max limit)
# These are trades between any participant, you may or may not have participated in these
trade_tick_history = exchange.get_trade_tick_history(instrument_id)

# Display last 5
trade_tick_history[-5:]

In [ ]:
# Poll all public tradeticks since the last time this method was called
exchange.poll_new_trade_ticks(instrument_id)

### Private trades

In [ ]:
# Load and store a list of all private trades you participated in since the instantiation of the Exchange (upto a max limit)
trade_history = exchange.get_trade_history(instrument_id)

# Display last 5
trade_history[-5:]

In [ ]:
# Poll all private trades since the last time this method was called
exchange.poll_new_trades(instrument_id)

### Position & PnL

In [ ]:
# Load current positions in all instruments
exchange.get_positions()

In [ ]:
# Load current positions in all instruments, including cash component (how much did we invest or gain by trading each instrument)
exchange.get_positions_and_cash()

In [ ]:
# Current PnL: sum of all cash components and value of the positions (evaluated against last-traded price) 
exchange.get_pnl()

# Code Snippets

Combining a few of the exchange interactions above, we can write a code snippet to aggressively trade out of all currently held positions using IOC orders. 

That means selling all products you are long, and buying back all you are short. 

This is useful for starting with a clean slate, but doing so will come at a potentially large cost. 

You are not guaranteed to end up with a zero position afterwards, only that volume which is also available to trade in the market will be bought/sold.

In [ ]:
MIN_SELLING_PRICE = 0.10
MAX_BUYING_PRICE = 100000.00

positions = exchange.get_positions()
pnl = exchange.get_pnl()

print(f'Positions before: {positions}')
print(f'\nPnL before: {pnl:.2f}')

print(f'\nTrading out of positions')
for iid, pos in positions.items():
    if pos > 0:
        print(f'-- Inserting sell order for {pos} lots of {iid}, with limit price {MIN_SELLING_PRICE:.2f}')
        exchange.insert_order(iid, price=MIN_SELLING_PRICE, volume=pos, side='ask', order_type='ioc')
    elif pos < 0:
        print(f'-- Inserting buy order for {abs(pos)} lots of {iid}, with limit price {MAX_BUYING_PRICE:.2f}')
        exchange.insert_order(iid, price=MAX_BUYING_PRICE, volume=-pos, side='bid', order_type='ioc')
    else:
        print(f'-- No initial position in {iid}, skipping..')
    
    time.sleep(0.10)

time.sleep(1.0)

positions = exchange.get_positions()
pnl = exchange.get_pnl()
print(f'\nPositions after: {positions}')
print(f'\nPnL after: {pnl:.2f}')